# Réseaux convolutionnels - Cifar 10

## Initialisations

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

In [ ]:
from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Activation, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

In [ ]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
# Permet d'éviter les erreurs mémoires pour le GPU
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## MNIST : modèle CNN simple

Pour utiliser Tensorflow, on doit reformatter l'image sous la forme **8x8x1** au lieu de **8x8** (pour ajouter le "canal", ici **1** puisque l'image est en niveaux de gris)

In [ ]:
plt.imshow(X_test[0].reshape(32,32,3))

Comme l'activation d'un neurone classe entre 0 ou 1, on code la cible (classes entre 0 et 9) sous la forme d'un vecteur de 0 ou 1 (*one hot encoding*) avec *to_categorical* :

In [ ]:
print(y_test[0])
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)
print(y_test[0])

In [ ]:
cifar_classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
print(cifar_classes)

In [ ]:
# Réseau dense simple
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(cifar_classes)))
model.add(Activation('softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


On peut afficher la structure du modèle :

In [ ]:
model.summary()

In [ ]:
# Apprentissage
train = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=200, verbose=1)

On peut afficher la structure du modèle :

In [ ]:
model.summary()

In [ ]:
# Test
scores = model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))

La variable *train* mémorise l'historique des scores sur l'ensemble d'apprentissage :

In [ ]:
print(train.history['accuracy'])

et sur l'ensemble de validation :

In [ ]:
print(train.history['val_accuracy'])

On définit une fonction pour afficher un graphique des scores :

In [ ]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

In [ ]:
# Prediction
y_cnn = model.predict_classes(X_test)

On obtient un vecteur de probabilités :

In [ ]:
print(y_test[0].argmax())

L'indice de la plus grande probabilité donne la classe prédite :

On peut afficher la matrice de confusion :

In [ ]:
cm = confusion_matrix(y_cnn,y_test)
print(cm)
plt.figure(figsize = (12,10))
sns.heatmap(cm, annot=True, cmap="coolwarm")

In [ ]:
import random

plt.figure(figsize=(15,25))
n_test = X_test.shape[0]
for i in range(1,50) :
    ir = random.randint(0,n_test)
    plt.subplot(10,5,i)
    plt.axis('off')
    plt.imshow(X_test[ir].reshape(32, 32, 3), cmap="gray_r")
    pred_classe = y_cnn[ir].argmax(axis=-1)
    plt.title('Calculated %s - Is %s' % (cifar_classes[y_cnn[ir]], cifar_classes[y_test[ir].argmax()]))

## Modèle CNN plus profond

Les images originales MNIST sont au format **28x28** (elles ont été redimensionnées pour le dataset de sklearn, afin de permettre une analyse plus facile avec les approches usuelles de machine learning). Keras dispose du dataset original :

In [ ]:
if False:
    model.save('cifar10.h5')

On peut ensuite utiliser le modèle sans recommencer l'entraînement :

In [ ]:
if False:
    new_model = load_model('cifar10.h5')
    new_model.summary()

In [ ]:
scores = new_model.evaluate(X_test, y_test, verbose=0)
print("Score : %.2f%%" % (scores[1]*100))